## SEGMENTATION

| **Step**                 | **Otsu Thresholding**                                             | **Active Contours**                                                      | **U-Net (Deep Learning)**                                                             |
| ------------------------ | ----------------------------------------------------------------- | ------------------------------------------------------------------------ | ------------------------------------------------------------------------------------- |
| **Initial Segmentation** | Global Otsu thresholding to binarize image                        | Evolve contour using Chan-Vese or morphological snakes from initial mask | Input image into trained U-Net to get breast mask                                     |
| **Pectoral Removal**     | Manual triangle masking or line detection (Hough)                 | Naturally excluded if contour initialized properly (top-left corner)     | Automatically learned during training (requires labeled data with no pectoral muscle) |
| **Postprocessing**       | - Fill holes<br>- Morph ops (open/close)<br>- Keep largest region | - Smooth final contour<br>- Apply contour as mask                        | - Minimal<br>- Optional morphological cleanup                                         |
| **Output**               | Binary mask with breast region                                    | Binary mask from active contour                                          | Binary mask from model prediction                                                     |
| **Advantages**           | Fast, simple, no training needed                                  | Good edge adherence, flexible, no training                               | Highest accuracy, robust to variation                                                 |
| **Limitations**          | Sensitive to noise, may include pectoral or miss weak tissue      | Needs proper initialization, slower                                      | Requires labeled data and training, heavier computational load                        |


In [28]:
# =============================================================================
# IMPORTS I CONSTANTS GLOBALS
# =============================================================================

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_multiotsu
from scipy.ndimage import binary_fill_holes
from skimage.filters import gabor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import torch
from model.unet_model import UNet

from pathlib import Path
from typing import Tuple, Dict, List, Callable

# CONSTANTS GLOBALS
BASE_PATH: str = "../data/test"
INPUT_FOLDER_PATHS: str = f"{BASE_PATH}/inputs/"
MASK_PATHS: Dict[str, str] = {
    'otsu': f"{BASE_PATH}/masks/otsu",
    'kmeans': f"{BASE_PATH}/masks/kmeans",
    'unet': f"{BASE_PATH}/masks/unet"
}

MODEL_PATH: str = "model/best_unet.pth"

In [29]:
# =============================================================================
# FUNCIONS PLANTILLA SIMPLES PER A SEGMENTACIÓ
# =============================================================================

def largest_connected_component(mask: np.ndarray) -> np.ndarray:
    """Troba i retorna el component connectat més gran de la màscara.
    
    Args:
        mask: Màscara binària d'entrada
        
    Returns:
        Màscara amb només el component connectat més gran
    """
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask)
    if num_labels <= 1:
        return mask
    largest: int = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    return (labels == largest).astype(np.uint8) * 255  # 0/255


def process_mask(mask: np.ndarray, open: int, close: int, size_k: int, lcc: bool = True) -> np.ndarray:
    """Processa una màscara amb operacions morfològiques.
    
    Args:
        mask: Màscara d'entrada
        open: Nombre d'iteracions d'obertura
        close: Nombre d'iteracions de tancament
        size_k: Mida del kernel morfològic
        lcc: Si cal mantenir només el component connectat més gran
        
    Returns:
        Màscara processada
    """
    kernel: np.ndarray = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size_k, size_k))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=open) 
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=close)
    if lcc: 
        mask = largest_connected_component(mask)
    return mask


def segmentation_otsu(imatge: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Segmenta una imatge en escala de grisos mitjançant Multi-Otsu, selecciona regions d'interès,
    omple concavitats grans i forats (fins i tot els que toquen la vora) i retorna la imatge segmentada
    i la màscara final suavitzada.
    
    Args:
        imatge: Imatge en escala de grisos
        
    Returns:
        Tupla amb (imatge_segmentada, màscara_pit)
    """

    # Multi-Otsu (4 classes)
    thresholds: np.ndarray = threshold_multiotsu(imatge, classes=4)
    regions: np.ndarray = np.digitize(imatge, bins=thresholds)
    

    # Selecció de classes rellevants (teixit fibrós i greix)
    fib_mask: np.ndarray = (regions == 1).astype(np.uint8) * 255
    fat_mask: np.ndarray = (regions == 2).astype(np.uint8) * 255

    # Fusió i neteja
    breast_mask: np.ndarray = cv2.bitwise_or(fib_mask, fat_mask)
    breast_mask = largest_connected_component(breast_mask)

    # Omplir forats interns (tancats i oberts a les vores)
    breast_mask = binary_fill_holes(breast_mask > 0).astype(np.uint8) * 255
    breast_mask = process_mask(breast_mask, open=1, close=3, size_k=5)

    # Suavitzar contorn
    breast_mask = cv2.GaussianBlur(breast_mask, (15, 15), 7)
    breast_mask = cv2.threshold(breast_mask, 120, 255, cv2.THRESH_BINARY)[1]
    breast_mask = binary_fill_holes(breast_mask > 0).astype(np.uint8) * 255

    # Aplicació de la màscara a la imatge original
    imatge_segmentada: np.ndarray = cv2.bitwise_and(imatge, imatge, mask=breast_mask)

    return imatge_segmentada, breast_mask


def segmentation_kmeans(imatge: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Segmenta una imatge utilitzant K-means clustering amb intensitat i textura Gabor (reduïda amb PCA).
    
    Args:
        imatge: Imatge en escala de grisos
        
    Returns:
        Tupla amb (imatge_segmentada, màscara_pit)
    """

    imatge_mod: np.ndarray = imatge.astype(np.float32)
    shape = imatge.shape

    def gabor_features(img: np.ndarray) -> np.ndarray:
        """
        Extreu textura amb diversos filtres Gabor i redueix amb PCA a una sola característica.
        """
        frequencies = [0.5, 0.6, 0.7, 0.8, 9.0, 1.0]
        thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]

        feature_list = []
        for freq in frequencies:
            for theta in thetas:
                response, _ = gabor(img, frequency=freq, theta=theta)
                feature_list.append(response.reshape(-1, 1))  # (n_pixels, 1)

        all_features = np.concatenate(feature_list, axis=1)  # (n_pixels, n_features)
 
        # Normalització i Reducció a 1 sola característica
        scaled = StandardScaler().fit_transform(all_features)
        texture_reduced = PCA(n_components=1).fit_transform(scaled)  # (n_pixels, 1)

        return texture_reduced

    # 1. Extreure intensitat i textura
    intensity = imatge_mod.reshape(-1, 1)
    texture = gabor_features(imatge_mod)

    # 2. Combinar
    features = np.concatenate([intensity, texture], axis=1)
    features_scaled = StandardScaler().fit_transform(features)

    # 3. K-means clustering
    K = 4
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, _ = cv2.kmeans(features_scaled.astype(np.float32), K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    labels = labels.flatten()
    clustered_image = labels.reshape(shape)

    # 4. Classificació de clusters
    mean_intensities = [np.mean(intensity[labels == k]) for k in range(K)]
    sorted_clusters = np.argsort(mean_intensities)
    tissue_clusters = {sorted_clusters[1], sorted_clusters[2]}  # entre fons i pectoral

    # 5. Màscara del pit
    breast_mask = np.isin(clustered_image, list(tissue_clusters)).astype(np.uint8) * 255
    breast_mask = process_mask(breast_mask, open=1, close=3, size_k=5, lcc=True)
    breast_mask = binary_fill_holes(breast_mask > 0).astype(np.uint8) * 255

    # 7. Segmentació final
    imatge_segmentada = cv2.bitwise_and(imatge, imatge, mask=breast_mask)

    return imatge_segmentada, breast_mask



def segmentation_unet(imatge: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Segmentació simple amb U-Net (simulació).
    Basat en: Entrada al model entrenat -> Predicció directa -> Postprocessament mínim
    
    Args:
        imatge: Imatge en escala de grisos
        
    Returns:
        Tupla amb (imatge_segmentada, màscara_pit)
    """
    input_size: Tuple[int, int] = (768, 512)  # (amplada, alçada)

    # ---------- Carregar Model ----------
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model: UNet = UNet().to(device)
    model.load_state_dict(torch.load(Path(MODEL_PATH), map_location=device))
    model.eval()

    # Convertir imatge a float32 i normalitzar a [0, 1]
    img_float: np.ndarray = imatge.astype(np.float32) / 255.0
    tensor: torch.Tensor = torch.tensor(img_float).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
        pred: torch.Tensor = model(tensor)
        breast_mask: np.ndarray = (pred.squeeze().cpu().numpy() > 0.5).astype(np.uint8) * 255

    breast_mask = process_mask(breast_mask, open=1, close=2, size_k=10, lcc=True)
    imatge_segmentada: np.ndarray = cv2.bitwise_and(imatge, imatge, mask=breast_mask)

    return imatge_segmentada, breast_mask

In [30]:
def process_all_images(input_folder: str, method: str = 'otsu') -> None:
    """Processa totes les imatges d'una carpeta d'entrada i les desa en una carpeta de sortida.
    
    Args:
        input_folder: Ruta de la carpeta d'entrada amb imatges
        method: Mètode de segmentació ('otsu', 'kmeans', 'unet')
    """
    input_path: Path = Path(input_folder)
    output_path: Path = Path(MASK_PATHS[method])

    segmentation_function: Dict[str, Callable[[np.ndarray], Tuple[np.ndarray, np.ndarray]]] = {
        'otsu': segmentation_otsu,
        'kmeans': segmentation_kmeans,
        'unet': segmentation_unet
    }

    print(f"Processant imatges a {input_path} utilitzant el mètode '{method}'...")

    for img_file in input_path.glob("*"):
        if img_file.suffix.lower() not in {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp"}:
            continue
        image: np.ndarray = cv2.imread(str(img_file), cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"❌ {img_file.name}")
            continue
        imatge_retallada: np.ndarray
        breast_mask: np.ndarray
        imatge_retallada, breast_mask = segmentation_function[method](image)
        
        save_path_img: Path = output_path / f"{img_file.stem}_img.tiff"
        save_path_msk: Path = output_path / f"{img_file.stem}_msk.tiff"
        cv2.imwrite(str(save_path_img), imatge_retallada)
        cv2.imwrite(str(save_path_msk), breast_mask)
        print(f"✓ {img_file.name}")


def main() -> None:
    """Funció principal que executa el processament d'imatges."""
    if not Path(INPUT_FOLDER_PATHS).exists():
        print("❌ Carpeta no trobada. ")
        Path(INPUT_FOLDER_PATHS).mkdir(parents=True)
        return

    # otsu, kmeans, unet are the methods available
    process_all_images(INPUT_FOLDER_PATHS, method='unet')

if __name__ == "__main__":
    main()

Processant imatges a ..\data\test\inputs utilitzant el mètode 'unet'...
✓ Original_1_b1_pib.tiff
✓ Original_1_b2_pib.tiff
✓ Original_1_b3_pib.tiff
✓ Original_1_b4_pib.tiff
